## Tugas 1


Terdapat dataset **mushroom**. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Memuat dataset
data = pd.read_csv('../../Data/mushrooms.csv')

# Preprocessing dataset
# Mengubah data kategorikal menjadi numerik
data = pd.get_dummies(data)

# Memisahkan fitur dan target
X = data.drop('class_e', axis=1)  # Ganti 'class_e' dengan nama kolom target yang sesuai
y = data['class_e']

# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tugas 1 - Decision Tree
dt = DecisionTreeClassifier()
dt_param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10]
}
dt_grid_search = GridSearchCV(dt, dt_param_grid, cv=5)
dt_grid_search.fit(X_train, y_train)

# Evaluasi Decision Tree
y_pred_dt = dt_grid_search.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Best parameters for Decision Tree: ", dt_grid_search.best_params_)
print("Decision Tree Test Accuracy: {:.2f}".format(acc_dt))

# Tugas 1 - Random Forest
rf = RandomForestClassifier()
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}
rf_grid_search = GridSearchCV(rf, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

# Evaluasi Random Forest
y_pred_rf = rf_grid_search.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Best parameters for Random Forest: ", rf_grid_search.best_params_)
print("Random Forest Test Accuracy: {:.2f}".format(acc_rf))


Best parameters for Decision Tree:  {'max_depth': None, 'min_samples_split': 2}
Decision Tree Test Accuracy: 1.00
Best parameters for Random Forest:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Random Forest Test Accuracy: 1.00


## Tugas 2


Terdapat dataset **mushroom**. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [10]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Tugas 2 - Decision Tree dengan AdaBoost
# Membuat AdaBoostClassifier dengan algoritma SAMME
ada = AdaBoostClassifier(algorithm='SAMME')

# Parameter grid untuk tuning hyperparameter
ada_param_grid = {
    'n_estimators': [50, 100, 200],
    'estimator': [DecisionTreeClassifier(max_depth=depth) for depth in [1, 2, 3]]
}

# Menggunakan GridSearchCV untuk menemukan parameter terbaik
ada_grid_search = GridSearchCV(ada, ada_param_grid, cv=5, n_jobs=-1)
ada_grid_search.fit(X_train, y_train)

# Evaluasi AdaBoost
y_pred_ada = ada_grid_search.predict(X_test)
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Best parameters for AdaBoost: ", ada_grid_search.best_params_)
print("AdaBoost Test Accuracy: {:.2f}".format(acc_ada))


Best parameters for AdaBoost:  {'estimator': DecisionTreeClassifier(max_depth=1), 'n_estimators': 50}
AdaBoost Test Accuracy: 1.00


## Tugas 3


Dengan menggunakan dataset **diabetes**, buatlah ensemble voting dengan algoritma

1. Logistic Regression

2. SVM kernel polynomial

3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

In [12]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Memuat dataset diabetes
diabetes = pd.read_csv('../../Data/diabetes.csv')

# Mengasumsikan kolom terakhir adalah target dan lainnya sebagai fitur
X = diabetes.iloc[:, :-1]  # Semua kolom kecuali kolom terakhir sebagai fitur
y = diabetes.iloc[:, -1]   # Kolom terakhir sebagai target

# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model
lr = LogisticRegression(max_iter=1000)
svm = SVC(kernel='poly', probability=True)
dt = DecisionTreeClassifier()

from sklearn.model_selection import GridSearchCV

# Parameter untuk Logistic Regression
param_lr = {'C': [0.01, 0.1, 1, 10]}

# Parameter untuk SVM
param_svm = {'degree': [2, 3, 4], 'C': [0.1, 1, 10]}

# Parameter untuk Decision Tree
param_dt = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 10, 20]}

# Tuning hyperparameter untuk setiap model bisa dilakukan seperti ini
grid_search_lr = GridSearchCV(lr, param_lr, cv=5)
grid_search_svm = GridSearchCV(svm, param_svm, cv=5)
grid_search_dt = GridSearchCV(dt, param_dt, cv=5)

# Fit untuk masing-masing model
grid_search_lr.fit(X_train, y_train)
grid_search_svm.fit(X_train, y_train)
grid_search_dt.fit(X_train, y_train)

# Ensemble Voting
voting_clf = VotingClassifier(estimators=[
    ('lr', lr),
    ('svm', svm),
    ('dt', dt)],
    voting='soft')  # 'soft' untuk mempertimbangkan probabilitas

# Melatih Voting Classifier
voting_clf.fit(X_train, y_train)

# Memprediksi dan evaluasi
y_pred_voting = voting_clf.predict(X_test)
acc_voting = accuracy_score(y_test, y_pred_voting)
print("Voting Classifier Test Accuracy: {:.2f}".format(acc_voting))

Voting Classifier Test Accuracy: 0.79
